In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import datetime
from datetime import timedelta, datetime
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from colorama import Fore, Back, Style


In [15]:
class BacktestResults:
    def __init__(self, df, cash, commission):
        self.df = df
        self.cash = cash
        self.commission = commission
        self.portfolio = [self.cash]
        self.bench_port = [self.cash]
        self.pnl = []
        self.pnl_up = []
        self.pnl_down = []
        self.pnl_net = [0]
        self.num_trades = 0
        self.long_trades = 0
        self.short_trades = 0
        self.fee_static = 0
        self.fee_comp = 0
        self.entries = {}
        self.exits = {}
        self.long_open_indices = []
        self.short_open_indices = []
        self.short_squareoff_indices = []
        self.long_squareoff_indices = []
        self.position = None
        self.bench_shares = self.cash / self.df['open'][0]
        
        for i in range(len(self.df)-1):
            signal = self.df['signals'].iloc[i]
            if self.portfolio[-1] <= 0:
                self.portfolio[-1] = 0
                break

            if signal == 1:  # Long signal
                if self.position is None:
                    entry = self.df['open'][i+1]
                    shares_comp = self.portfolio[-1] / entry
                    shares_static = self.cash / entry
                    self.position = 'Long'
                    self.long_open_indices.append(i+1)
                    self.entries[i] = entry

            elif signal == -1:  # Short signal
                if self.position == 'Long':
                    exit = self.df['open'][i+1]
                    pnl_comp = shares_comp * (exit - entry) - self.commission * shares_comp * (entry + exit)
                    pnl_static = shares_static * (exit - entry) - self.commission * shares_static * (entry + exit)
                    self.portfolio.append(self.portfolio[-1] + pnl_comp)
                    self.pnl.append(pnl_static)
                    self.pnl_net.append(self.pnl_net[-1] + pnl_static)
                    self.num_trades += 1
                    self.long_trades += 1
                    
                    # Record PnL
                    if pnl_static > 0:
                        self.pnl_up.append(pnl_static)
                    else:
                        self.pnl_down.append(pnl_static)

                    self.position = None
                    self.long_squareoff_indices.append(i+1)
                    self.exits[i] = exit
                    self.bench_port.append(self.bench_shares * exit)

                elif self.position is None:
                    entry = self.df['open'][i+1]
                    shares_comp = self.portfolio[-1] / entry
                    shares_static = self.cash / entry
                    self.position = 'Short'
                    self.short_open_indices.append(i+1)
                    self.entries[i] = entry

            # Handle exit conditions for short positions
            if self.position == 'Short' and signal == 1:
                exit = self.df['open'][i+1]
                pnl_comp = shares_comp * (entry - exit) - self.commission * shares_comp * (entry + exit)
                pnl_static = shares_static * (entry - exit) - self.commission * shares_static * (entry + exit)
                self.portfolio.append(self.portfolio[-1] + pnl_comp)
                self.pnl.append(pnl_static)
                self.pnl_net.append(self.pnl_net[-1] + pnl_static)
                self.num_trades += 1
                self.short_trades += 1
                
                # Record PnL
                if pnl_static > 0:
                    self.pnl_up.append(pnl_static)
                else:
                    self.pnl_down.append(pnl_static)

                self.position = None
                self.short_squareoff_indices.append(i+1)
                self.exits[i] = exit
                self.bench_port.append(self.bench_shares * exit)

# ... existing methods remain unchanged ...


        
    def trade_history(self):
        print("--- TRADE HISTORY ---")
        print()

        position = None
        
        for i in range(len(self.df)-1):
            if self.df['signals'][i] == 1:
                if position == None: 
                    print("LONG POSITION")
                    print(self.df['datetime'][i+1], ":", "ENTRY - BUY", "@", self.df['open'][i+1])
                    position = 'Long'
                    
                elif position == 'Short':
                    print(self.df['datetime'][i+1], ":", "EXIT - BUY", "@", self.df['open'][i+1])
                    print()
                    position = None
                
            elif self.df['signals'][i] == -1:
                if position == 'Long':
                    print(self.df['datetime'][i+1], ":", "EXIT - SELL", "@", self.df['open'][i+1]) 
                    print()
                    position = None
                    
                elif position == None:
                    print("SHORT POSITION")
                    print(self.df['datetime'][i+1], ":", "ENTRY - SELL", "@", self.df['open'][i+1])
                    position = 'Short'
        
    def trade_metrics(self):
        entry_times = [key for key in self.entries.keys()]
        exit_times = [key for key in self.exits.keys()]
        trade_durations = []
        for i in range(len(entry_times)):
            trade_durations.append(exit_times[i] - entry_times[i])
        max_duration = np.max(trade_durations)
        max_duration_days = max_duration // 24
        max_duration_hours = max_duration % 24
        avg_duration = np.round(np.mean(trade_durations), 0)
        avg_duration_days = avg_duration // 24
        avg_duration_hours = avg_duration % 24
        print("Maximum Trade Duration:", max_duration_days, "days", max_duration_hours, "hours")
        print("Average Trade Duration:", avg_duration_days, "days", avg_duration_hours, "hours")
        print("Number of Trades:", self.num_trades)
        print("Winning Trades:", len(self.pnl_up))
        print("Losing Trades:", len(self.pnl_down))
        print("Win Rate:", np.round(100 * len(self.pnl_up) / len(self.pnl), 3), "%")
        print("Number of Long Trades:", self.long_trades)
        print("Number of Short Trades:", self.short_trades)
        
    def compounded_metrics(self):
        # Ensure datetime column is in the correct format
        self.df['datetime'] = pd.to_datetime(self.df['datetime'])

        # Calculate net return
        net_return = (self.portfolio[-1] / self.portfolio[0]) - 1
        
        # Calculate the number of years based on the length of the DataFrame
        num_years = (self.df['datetime'].iloc[-1] - self.df['datetime'].iloc[0]).days / 365.0
        
        # Annualized return calculation
        annual_return = (1 + net_return) ** (1 / num_years) - 1 if num_years > 0 else 0
        
        # Buy and hold return calculation
        bnh_return = (self.df['close'].iloc[-1] / self.df['close'].iloc[0]) - 1
        
        # Drawdown calculations
        dd = 1 - self.portfolio / np.maximum.accumulate(self.portfolio)
        max_draw = -np.nan_to_num(dd.max())
        
        print("--- COMPOUNDING ---")
        print("Initial Balance:", np.round(self.portfolio[0], 3))
        print("Final Balance:", np.round(self.portfolio[-1], 3))
        print("Peak Balance:", np.round(np.max(self.portfolio), 3))
        print("Maximum Dip:", -np.round(np.max(np.maximum.accumulate(self.portfolio) - self.portfolio), 3))
        print("Average Dip:", -np.round(np.mean(np.maximum.accumulate(self.portfolio) - self.portfolio), 3))
        print("Lowest Balance:", np.round(np.min(self.portfolio), 3))
        print("Maximum PnL:", np.round(np.diff(self.portfolio).max(), 3))
        print("Minimum PnL:", np.round(np.diff(self.portfolio).min(), 3))
        print("Net Return:", np.round(100 * net_return, 3), "%")
        print("Annualised Return:", np.round(100 * annual_return, 3), "%")
        print("Buy and Hold Return (of BTC/USDT):", np.round(100 * bnh_return, 3), "%")
        print("Maximum Drawdown:", np.round(100 * max_draw, 3), "%")
        print("Total Transaction Fees:", np.round(self.fee_comp, 3))
            
    def static_metrics(self):
        bench = (self.df['close'].iloc[-1] / self.df['close'].iloc[0] - 1) * self.cash
        entry_times = [key for key in self.entries.keys()]
        exit_times = [key for key in self.exits.keys()]
        trade_durations = []
        for i in range(len(entry_times)):
            trade_durations.append(exit_times[i] - entry_times[i])
        dur = np.array(trade_durations) / 24
        ret = np.array(self.pnl) / 1000
        daily_ret = ret / dur
        neg_ret = daily_ret[daily_ret < 0]
        sharpe = (daily_ret.mean() * 365) / (daily_ret.std() * np.sqrt(365)) if daily_ret.std() != 0 else 0
        sortino = (daily_ret.mean() * 365) / (neg_ret.std() * np.sqrt(365)) if neg_ret.std() != 0 else 0

        print("--- STATIC ---")
        print("Capital per Trade:", self.cash)
        print("Net Profit:", np.round(self.pnl_net[-1], 3))
        print("Gross Profit:", np.round(self.pnl_net[-1] + self.fee_static, 3))
        print("Benchmark Return:", np.round(bench, 3))
        print("Sharpe Ratio:", np.round(sharpe, 3))
        print("Sortino Ratio:", np.round(sortino, 3))
        print("Average Profit/Loss per Trade:", np.round(np.mean(self.pnl), 3))

        # Check for empty pnl_up and pnl_down
        if len(self.pnl_up) > 0:
            print("Largest Win:", np.round(np.max(self.pnl_up), 3))
            print("Average Win:", np.round(np.mean(self.pnl_up), 3))
        else:
            print("Largest Win: N/A (No winning trades)")
            print("Average Win: N/A (No winning trades)")

        if len(self.pnl_down) > 0:
            print("Largest Loss:", np.round(np.min(self.pnl_down), 3))
            print("Average Loss:", np.round(np.mean(self.pnl_down), 3))
        else:
            print("Largest Loss: N/A (No losing trades)")
            print("Average Loss: N/A (No losing trades)")

        print("Total Transaction Fees:", np.round(self.fee_static, 3))
        
        
    def plot_equity_curves(self):    
        fig = make_subplots(rows=2, cols=1, subplot_titles=["Static Position", "Compounded"])
        fig.add_trace(px.line(y=self.pnl_net).data[0], row=1, col=1)
        fig.add_trace(px.line(y=self.portfolio).data[0], row=2, col=1)
        fig.update_xaxes(title_text="# of Trades", row=1, col=1)
        fig.update_xaxes(title_text="# of Trades", row=2, col=1)
        fig.update_yaxes(title_text="Net Profit/Loss", row=1, col=1)
        fig.update_yaxes(title_text="Portfolio Balance", row=2, col=1)
        fig.update_layout(template="ggplot2")
        fig.update_layout(height = 900, width = 900)
        fig.show()
        
    def plot_drawdown_curve(self):        
        dd = 100 * np.round(-(1 - self.portfolio / np.maximum.accumulate(self.portfolio)), 3)
        fig = px.bar(x=list(range(len(dd))), y=dd ,
             title='Portfolio Drawdown',
             height=600)
        fig.update_xaxes(title_text="# of Trades")
        fig.update_yaxes(title_text="Portfolio Drawdown (%)")
        fig.update_layout(template="ggplot2")
        fig.update_layout(coloraxis_colorbar=dict(title='Portfolio Drawdown (%)'))
        fig.update_layout(yaxis=dict())
        fig.show()
        
    def plot_long_trades(self):
        fig = px.line(self.df, y="open", x=self.df["datetime"], title='Long Trades')
        long_open_dates = []
        long_open_prices = []
        long_close_dates = []
        long_close_prices = []
        for i in range(len(self.long_open_indices)):
            long_open_dates.append(self.df["datetime"][self.long_open_indices[i]])
            long_open_prices.append(self.df["open"][self.long_open_indices[i]])
        for i in range(len(self.long_squareoff_indices)):
            long_close_dates.append(self.df["datetime"][self.long_squareoff_indices[i]])
            long_close_prices.append(self.df["open"][self.long_squareoff_indices[i]])
        fig.add_trace(go.Scatter(x=long_open_dates, y=long_open_prices, mode='markers', marker=dict(size=20, color='green',symbol="triangle-up"), name="Long Entry"))
        fig.add_trace(go.Scatter(x=long_close_dates, y=long_close_prices, mode='markers', marker=dict(size=20, color='blue',symbol="triangle-down"), name="Long Exit"))
        fig.update_layout(template="ggplot2")
        fig.update_xaxes(title_text="Open")
        fig.show()
        
    def plot_short_trades(self):
        fig = px.line(self.df, y="open", x=self.df["datetime"], title='Short Trades')
        short_open_dates = []
        short_open_prices = []
        short_close_dates = []
        short_close_prices = []
        for i in range(len(self.short_open_indices)):
            short_open_dates.append(self.df["datetime"][self.short_open_indices[i]])
            short_open_prices.append(self.df["open"][self.short_open_indices[i]])
        for i in range(len(self.short_squareoff_indices)):
            short_close_dates.append(self.df["datetime"][self.short_squareoff_indices[i]])
            short_close_prices.append(self.df["open"][self.short_squareoff_indices[i]])
        fig.add_trace(go.Scatter(x=short_open_dates, y=short_open_prices, mode='markers', marker=dict(size=20, color='green',symbol="triangle-down"), name="Short Entry"))
        fig.add_trace(go.Scatter(x=short_close_dates, y=short_close_prices, mode='markers', marker=dict(size=20, color='blue',symbol="triangle-up"), name="Short Exit"))
        fig.update_layout(template="ggplot2")
        fig.update_xaxes(title_text="Open")
        fig.show()
    

In [16]:
df_sub = pd.read_csv("btc_strategy_results.csv")
res = BacktestResults(df_sub, cash = 1000.0, commission = 0.0015)

In [17]:
# Drop all columns except those needed for backtesting
columns_to_keep = ['datetime', 'signals', 'open', 'high', 'low', 'close', 'volume']
df_sub = df_sub[columns_to_keep]

In [18]:
df_sub.columns

Index(['datetime', 'signals', 'open', 'high', 'low', 'close', 'volume'], dtype='object')

In [19]:
df_sub['signals'].value_counts()


signals
 0    8686
 1      31
-1      29
-2       1
Name: count, dtype: int64

In [20]:
res.trade_history()

--- TRADE HISTORY ---

LONG POSITION
2023-01-03 18:00:00 : ENTRY - BUY @ 16626.1
2023-01-12 06:00:00 : EXIT - SELL @ 18080.6

LONG POSITION
2023-01-12 18:00:00 : ENTRY - BUY @ 18840.0
2023-02-02 02:00:00 : EXIT - SELL @ 23958.6

LONG POSITION
2023-02-02 15:00:00 : ENTRY - BUY @ 23612.1
2023-02-09 20:00:00 : EXIT - SELL @ 22027.3

LONG POSITION
2023-02-10 11:00:00 : ENTRY - BUY @ 21841.7
2023-03-08 06:00:00 : EXIT - SELL @ 21944.2

LONG POSITION
2023-03-08 07:00:00 : ENTRY - BUY @ 22015.0
2023-03-09 20:00:00 : EXIT - SELL @ 20845.6

LONG POSITION
2023-03-11 03:00:00 : ENTRY - BUY @ 20479.9
2023-03-13 02:00:00 : EXIT - SELL @ 22280.8

LONG POSITION
2023-03-13 09:00:00 : ENTRY - BUY @ 22185.3
2023-03-13 21:00:00 : EXIT - SELL @ 24160.4

LONG POSITION
2023-03-14 12:00:00 : ENTRY - BUY @ 24747.4
2023-03-14 18:00:00 : EXIT - SELL @ 25651.3

LONG POSITION
2023-03-15 14:00:00 : ENTRY - BUY @ 24799.9
2023-03-22 19:00:00 : EXIT - SELL @ 28018.9

LONG POSITION
2023-03-22 23:00:00 : ENTRY - BUY @ 

In [21]:
res.trade_metrics()

IndexError: list index out of range

In [22]:
res.static_metrics()

IndexError: list index out of range

In [23]:
res.compounded_metrics()

--- COMPOUNDING ---
Initial Balance: 1000.0
Final Balance: 2858.994
Peak Balance: 2858.994
Maximum Dip: -257.375
Average Dip: -61.066
Lowest Balance: 1000.0
Maximum PnL: 482.064
Minimum PnL: -154.451
Net Return: 185.899 %
Annualised Return: 186.726 %
Buy and Hold Return (of BTC/USDT): 157.081 %
Maximum Drawdown: -14.164 %
Total Transaction Fees: 0


In [24]:
res.plot_equity_curves()

In [25]:
res.plot_long_trades()

In [26]:
res.plot_short_trades()

In [27]:
res.plot_drawdown_curve()

In [15]:
# from untrade.client import Client
# from pprint import pprint

# def perform_backtest(csv_file_path):
#     # Create an instance of the untrade client
#     client = Client()
#     # Perform backtest using the provided CSV file path
#     result = client.backtest(
#         file_path=csv_file_path,
#         leverage=1,  # Adjust leverage as needed
#         jupyter_id="abhishekd.7006",  # the one you use to login to jupyter.untrade.io
#     )
#     return result

# if __name__ == "__main__":
#     csv_file_path = "corr_strategy_results.csv"
#     backtest_result = perform_backtest(csv_file_path)
#     for value in backtest_result:
#         print(value)



In [16]:
# for value in backtest_result:        
#     print(value)